In [2]:
# Imports and functions
%matplotlib inline
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,Input, Flatten
from keras import models
from keras import optimizers
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
nuc_arr = ['A','C','G','T']
#Function for calculating modified probability of splicing at SD1, only considering SD1 and SD2
def prob_SD1 (sd1_freq, sd2_freq): 
    if (sd1_freq==0 and sd2_freq==0):
        return 0.0 
    else:
        return sd1_freq/(sd1_freq+sd2_freq)
def prob_site (site_freq, total_freq):
    if (total_freq==0):
        return 0.0
    else:
        return float(site_freq)/total_freq
#Function converting nucleotide sequence to numerical array with 4 channels
def seq_to_arr (seq):
    seq_len = len(seq)
    arr_rep = np.zeros((seq_len, len(nuc_arr))) 
    for i in range(seq_len):
        arr_rep[i][nuc_arr.index(seq[i])] = 1 
    return arr_rep

Using TensorFlow backend.


In [12]:
#Creating a modified dataset with only the necessary information
#Storing model inputs and outputs
reads_path = 'GSM1911084_A3SS_spliced_reads.txt'
seq_path = 'GSM1911083_A3SS_seq.txt'
s1_indx = 236
s2_indx= 389
seq_len = 80
read_lines = []
seq_lines = []
data_table = []

#Getting all sequences
with open(seq_path) as f:
    f.readline() 
    for line in f:
        mod_line = line.split('\t') 
        seq_lines.append([mod_line[0], mod_line[1][:-1]])
        
#Getting all relevant read frequencies
with open(reads_path) as f:
    f.readline() 
    for line in f:
        mod_line = line.split('\t')
        read_lines.append([mod_line[0], mod_line[s1_indx], mod_line[s2_indx]])
        
n = len(read_lines)
prob_s1 = np.zeros(n)
inputs = np.zeros((n,seq_len, 4))

with open('3SS_compressed.txt', 'w') as f: 
    for i in range(n):
        data_table.append([read_lines[i][0], seq_lines[i][1], read_lines[i][1], read_lines[i][2]])
        f.write(read_lines[i][0]+'\t'+seq_lines[i][1]+'\t'+read_lines[i][1]+ '\t'+read_lines[i][2] + '\n')
        prob_s1[i] = prob_SD1(float(read_lines[i][1]), float(read_lines[i][2]))
        inputs[i] = seq_to_arr(seq_lines[i][1])

In [20]:
#Creating, training, and testing model_1, a somewhat simpler model
#Architecture 1: Input -> Conv -> Pool -> Conv -> Pool -> FC -> FC
model_1 = models.Sequential([
    Conv1D(seq_len//2,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    MaxPooling1D(pool_size=3),
    Conv1D(seq_len//4, (4), strides=1, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten(),
    Dense(25, activation='relu'),
    Dense(1)
])
#Compiling and training
model_1.compile(optimizer=optimizers.Adam(lr=0.001), loss='mean_squared_error')
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(inputs, prob_s1,test_size=0.1, random_state=40) 
model_1.fit(x=X_train_1, y=y_train_1, epochs=2) 
#Testing
y_pred_1 = model_1.predict(X_test_1) 
print("Model 1 r2 on Testing Data:") 
print(r2_score(y_test_1, y_pred_1))

Epoch 1/2
1990565/1990565 [==============================] - 229s 115us/step - loss: 0.0177
Epoch 2/2
1990565/1990565 [==============================] - 229s 115us/step - loss: 0.0174
Model 1 r2 on Testing Data:
0.11747698356624015


In [21]:
#Creating, training, and testing model_2, a slightly deeper network
#Architecture 2: Input -> Conv -> Conv -> Pool -> Conv -> Conv -> Pool -> FC -> FC
model_2 = models.Sequential([
    Conv1D(seq_len,(4), input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    Conv1D(seq_len,(4), input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    MaxPooling1D(pool_size=5),
    Conv1D(seq_len//2,(4), input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    Conv1D(seq_len//2,(4), input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    MaxPooling1D(pool_size=5),
    Flatten(),
    Dense(40, activation='relu'),
    Dense(1)
])
#Compiling and training
model_2.compile(optimizer=optimizers.Adam(lr=0.001), loss='mean_squared_error')
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(inputs, prob_s1,test_size=0.1, random_state=40) 
model_2.fit(x=X_train_2, y=y_train_2, epochs=2) 
#Testing
y_pred_2 = model_2.predict(X_test_2) 
print("Model 2 r2 on Testing Data:") 
print(r2_score(y_test_2, y_pred_2))

Epoch 1/2
1990565/1990565 [==============================] - 369s 185us/step - loss: 0.0185
Epoch 2/2
1990565/1990565 [==============================] - 365s 183us/step - loss: 0.0183
Model 2 r2 on Testing Data:
0.07737825372849572


In [3]:
#BELOW, MODELS ARE TRAINED TO PREDICT TOTAL READ FRACTION AT SA1, SA2, Crypt, not Binary Fraction
#Creating a modified dataset with only the necessary information
#Storing model inputs and outputs
reads_path = 'GSM1911084_A3SS_spliced_reads.txt'
seq_path = 'GSM1911083_A3SS_seq.txt'
s1_indx = 236
s2_indx= 389
crypt_indx = 373
seq_len = 80
read_lines = []
seq_lines = []

#Getting all sequences
with open(seq_path) as f:
    f.readline() 
    for line in f:
        mod_line = line.split('\t') 
        seq_lines.append([mod_line[0], mod_line[1][:-1]])
        
#Getting all relevant read frequencies
with open(reads_path) as f:
    f.readline() 
    for line in f:
        mod_line = line.split('\t')
        total_reads = sum([int(j) for j in mod_line[1:]])
        read_lines.append([mod_line[0], prob_site (mod_line[s1_indx], total_reads), prob_site (mod_line[s2_indx], total_reads), prob_site (mod_line[crypt_indx], total_reads)])

n = len(read_lines)
prob_s1 = np.zeros(n)
prob_s2 = np.zeros(n)
prob_cr = np.zeros(n)
inputs = np.zeros((n,seq_len, 4))

for i in range(n):
    prob_s1[i] = float(read_lines[i][1])
    prob_s2[i] = float(read_lines[i][2])
    prob_cr[i] = float(read_lines[i][3])
    inputs[i] = seq_to_arr(seq_lines[i][1])


In [5]:
#SA1 Prediction
#Creating, training, and testing model_1, a somewhat simpler model
#Architecture 1: Input -> Conv -> Pool -> Conv -> Pool -> FC -> FC
model_1 = models.Sequential([
    Conv1D(seq_len//2,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    MaxPooling1D(pool_size=3),
    Conv1D(seq_len//4, (4), strides=1, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten(),
    Dense(25, activation='relu'),
    Dense(1)
])
model_1.compile(optimizer=optimizers.Adam(lr=0.001), loss='mean_squared_error')
#Training Model 1
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(inputs, prob_s1,test_size=0.1, random_state=40) 
model_1.fit(x=X_train_1, y=y_train_1, epochs=2) 
#Testing
y_pred_1_tr = model_1.predict(X_train_1)
print("Model 1 r2 on Training Data for SA1:")
print(r2_score(y_train_1, y_pred_1_tr))
y_pred_1 = model_1.predict(X_test_1) 
print("Model 1 r2 on Testing Data for SA1:") 
print(r2_score(y_test_1, y_pred_1))

#Creating, training, and testing model_2, a slightly deeper network
#Architecture 2: Input -> Conv -> Conv -> Pool -> Conv -> Conv -> Pool -> FC -> FC
model_2 = models.Sequential([
    Conv1D(seq_len,(4), input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    Conv1D(seq_len,(4), input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    MaxPooling1D(pool_size=5),
    Conv1D(seq_len//2,(4), input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    Conv1D(seq_len//2,(4), input_shape=(seq_len,len(nuc_arr)), activation='relu'),
    MaxPooling1D(pool_size=5),
    Flatten(),
    Dense(40, activation='relu'),
    Dense(1)
])
model_2.compile(optimizer=optimizers.Adam(lr=0.001), loss='mean_squared_error')
#Training Model 2
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(inputs, prob_s1,test_size=0.1, random_state=40) 
model_2.fit(x=X_train_2, y=y_train_2, epochs=2) 
#Testing
y_pred_2_tr = model_2.predict(X_train_2)
print("Model 2 r2 on Training Data for SA1:")
print(r2_score(y_train_2, y_pred_2_tr))
y_pred_2 = model_2.predict(X_test_2) 
print("Model 2 r2 on Testing Data for SA1:") 
print(r2_score(y_test_2, y_pred_2))

Epoch 1/2
1990565/1990565 [==============================] - 228s 114us/step - loss: 0.0169
Epoch 2/2
1990565/1990565 [==============================] - 228s 115us/step - loss: 0.0166
Model 1 r2 on Training Data for SA1:
0.12484327987776656
Model 1 r2 on Testing Data for SA1:
0.1219766311251902
Epoch 1/2
1990565/1990565 [==============================] - 354s 178us/step - loss: 0.0176
Epoch 2/2
1990565/1990565 [==============================] - 352s 177us/step - loss: 0.0174
Model 2 r2 on Training Data for SA1:
0.07072624372190128
Model 2 r2 on Testing Data for SA1:
0.0670908755481806


In [6]:
#SA2 Prediction
#Creating, training, and testing model_1, a somewhat simpler model
#Architecture 1: Input -> Conv -> Pool -> Conv -> Pool -> FC -> FC

#Training Model 1
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(inputs, prob_s2,test_size=0.1, random_state=40) 
model_1.fit(x=X_train_1, y=y_train_1, epochs=2) 
#Testing Model 1
y_pred_1_tr = model_1.predict(X_train_1)
print("Model 1 r2 on Training Data for SA2:")
print(r2_score(y_train_1, y_pred_1_tr))
y_pred_1 = model_1.predict(X_test_1) 
print("Model 1 r2 on Testing Data for SA2:") 
print(r2_score(y_test_1, y_pred_1))

#Creating, training, and testing model_2, a slightly deeper network
#Architecture 2: Input -> Conv -> Conv -> Pool -> Conv -> Conv -> Pool -> FC -> FC
#Training Model 2
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(inputs, prob_s2,test_size=0.1, random_state=40) 
model_2.fit(x=X_train_2, y=y_train_2, epochs=2) 
#Testing Model 2
y_pred_2_tr = model_2.predict(X_train_2)
print("Model 2 r2 on Training Data for SA2:")
print(r2_score(y_train_2, y_pred_2_tr))
y_pred_2 = model_2.predict(X_test_2) 
print("Model 2 r2 on Testing Data for SA2:") 
print(r2_score(y_test_2, y_pred_2))

Epoch 1/2
1990565/1990565 [==============================] - 231s 116us/step - loss: 0.1754
Epoch 2/2
1990565/1990565 [==============================] - 234s 118us/step - loss: 0.1746
Model 1 r2 on Training Data for SA2:
0.0881002651992695
Model 1 r2 on Testing Data for SA2:
0.08494280263877951
Epoch 1/2
1990565/1990565 [==============================] - 359s 180us/step - loss: 0.1833
Epoch 2/2
1990565/1990565 [==============================] - 358s 180us/step - loss: 0.1816
Model 2 r2 on Training Data for SA2:
0.052681205761415995
Model 2 r2 on Testing Data for SA2:
0.050646736380798196


In [7]:
#Crypt Prediction
#Creating, training, and testing model_1, a somewhat simpler model
#Architecture 1: Input -> Conv -> Pool -> Conv -> Pool -> FC -> FC

#Training Model 1
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(inputs, prob_cr,test_size=0.1, random_state=40) 
model_1.fit(x=X_train_1, y=y_train_1, epochs=2) 
#Testing Model 1
y_pred_1_tr = model_1.predict(X_train_1)
print("Model 1 r2 on Training Data for SACrypt:")
print(r2_score(y_train_1, y_pred_1_tr))
y_pred_1 = model_1.predict(X_test_1) 
print("Model 1 r2 on Testing Data for SACrypt:") 
print(r2_score(y_test_1, y_pred_1))

#Training Model 2
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(inputs, prob_cr,test_size=0.1, random_state=40) 
model_2.fit(x=X_train_2, y=y_train_2, epochs=2) 
#Testing Model 2
y_pred_2_tr = model_2.predict(X_train_2)
print("Model 2 r2 on Training Data for SACrypt:")
print(r2_score(y_train_2, y_pred_2_tr))
y_pred_2 = model_2.predict(X_test_2) 
print("Model 2 r2 on Testing Data for SACrypt:") 
print(r2_score(y_test_2, y_pred_2))

Epoch 1/2
1990565/1990565 [==============================] - 233s 117us/step - loss: 0.0194
Epoch 2/2
1990565/1990565 [==============================] - 232s 117us/step - loss: 0.0191
Model 1 r2 on Training Data for SACrypt:
0.0008624130192416146
Model 1 r2 on Testing Data for SACrypt:
0.00080778351508215
Epoch 1/2
1990565/1990565 [==============================] - 361s 181us/step - loss: 0.0193
Epoch 2/2
1990565/1990565 [==============================] - 361s 182us/step - loss: 0.0192
Model 2 r2 on Training Data for SACrypt:
0.0002886424975629964
Model 2 r2 on Testing Data for SACrypt:
0.0002489518243189437
